In [4]:

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
import optuna
from sklearn.preprocessing import LabelEncoder 
from sksurv.metrics import concordance_index_censored
import bisect

In [5]:
data = pd.read_csv('train.csv')

X = data.drop(['efs_time', 'efs'], axis=1) 
y_time = data['efs_time']
y_event = data['efs']

numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# X[numerical_cols] = X[numerical_cols].fillna(X[numerical_cols].mean())
# X[categorical_cols] = X[categorical_cols].fillna(X[categorical_cols].mode().iloc[0])
# le_dict = {}
# for col in categorical_cols:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col])
#     le_dict[col] = le

# Handle categorical columns
for col in categorical_cols:
    X[col] = X[col].astype('object')
    value_counts = X[col].value_counts()
    category_map = {cat: idx for idx, cat in enumerate(value_counts.index)}
    X[col] = X[col].map(category_map)

In [6]:
X

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,0,3.0,0.0,NaN,0.0,NaN,NaN,0,0.0,6.0,...,90.0,0.0,NaN,1.0,0.0,8.0,0.0,2.0,0.0,10.0
1,1,0.0,0.0,1.0,0.0,2.0,8.0,3,0.0,6.0,...,90.0,0.0,0.0,0.0,0.0,8.0,0.0,2.0,1.0,10.0
2,2,3.0,0.0,NaN,0.0,2.0,8.0,0,0.0,6.0,...,90.0,0.0,0.0,0.0,0.0,8.0,0.0,2.0,0.0,10.0
3,3,2.0,0.0,1.0,0.0,2.0,8.0,0,0.0,6.0,...,90.0,1.0,0.0,1.0,0.0,8.0,0.0,2.0,0.0,10.0
4,4,2.0,0.0,NaN,0.0,2.0,8.0,0,0.0,6.0,...,90.0,0.0,0.0,0.0,1.0,8.0,0.0,2.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,28795,7.0,NaN,2.0,0.0,2.0,8.0,0,0.0,6.0,...,NaN,NaN,3.0,NaN,0.0,8.0,NaN,2.0,0.0,10.0
28796,28796,2.0,0.0,0.0,1.0,1.0,4.0,0,0.0,5.0,...,90.0,0.0,1.0,0.0,0.0,6.0,1.0,1.0,1.0,8.0
28797,28797,4.0,NaN,0.0,NaN,2.0,8.0,0,NaN,6.0,...,90.0,NaN,1.0,1.0,0.0,8.0,NaN,2.0,0.0,10.0
28798,28798,3.0,0.0,0.0,0.0,1.0,4.0,0,0.0,3.0,...,90.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,5.0


In [7]:
class CoxObjective:
    def __init__(self, time, event):
        self.time = time
        self.event = event
        self.sorted_indices = np.argsort(self.time)
        self.sorted_time = self.time[self.sorted_indices]
        self.sorted_event = self.event[self.sorted_indices]
        self.event_positions_sorted = [i for i in range(len(self.sorted_event)) if self.sorted_event[i] == 1]

    def __call__(self, y_pred, dataset=None):  # Added dataset parameter with default None
        sorted_exp_f = np.exp(y_pred[self.sorted_indices])
        sum_risk = np.cumsum(sorted_exp_f[::-1])[::-1]
        
        sum_1_over_sum_risk = []
        sum_1_over_sum_risk_squared = []
        for event_pos in self.event_positions_sorted:
            sum_r = sum_risk[event_pos]
            sum_1_over_sum_risk.append(1 / sum_r)
            sum_1_over_sum_risk_squared.append(1 / sum_r**2)
        
        cum_sum_1_over_sum_risk = np.cumsum(sum_1_over_sum_risk)
        cum_sum_1_over_sum_risk_squared = np.cumsum(sum_1_over_sum_risk_squared)
        
        gradients = np.zeros(len(y_pred))
        hessians = np.zeros(len(y_pred))
        sorted_position = {self.sorted_indices[k]: k for k in range(len(self.sorted_indices))}
        
        for k in range(len(y_pred)):
            sorted_pos_k = sorted_position[k]
            index = bisect.bisect_right(self.event_positions_sorted, sorted_pos_k)
            sum_1_over_sum_risk_k = cum_sum_1_over_sum_risk[index-1] if index > 0 else 0
            sum_1_over_sum_risk_squared_k = cum_sum_1_over_sum_risk_squared[index-1] if index > 0 else 0
            
            exp_f_k = np.exp(y_pred[k])
            sum_term_grad = exp_f_k * sum_1_over_sum_risk_k
            sum_term_hess = exp_f_k * sum_1_over_sum_risk_k - exp_f_k**2 * sum_1_over_sum_risk_squared_k
            
            gradient_k = - self.event[k] + sum_term_grad
            hessian_k = sum_term_hess
            
            gradients[k] = gradient_k
            hessians[k] = hessian_k
            
        return gradients, hessians

def objective(trial):
    
    params = {
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 25, 127),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-06, 10., log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-06, 10., log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'verbose': -1,
        'num_threads': 4,
        'seed': 42
    }
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    race_groups = X['race_group'].unique()
    fold_scores = []
    fold = -1

    for train_idx, val_idx in skf.split(X, X['race_group']):
        fold += 1
        print(f'Running in fold {fold}...')
        
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_time_train, y_time_val = y_time.iloc[train_idx], y_time.iloc[val_idx]
        y_event_train, y_event_val = y_event.iloc[train_idx], y_event.iloc[val_idx]

        train_data = lgb.Dataset(X_train, label= y_time_train)
        val_data = lgb.Dataset(X_val, label= y_time_val)

        def cindex_eval(y_pred, data_val):
            # Calculate stratified C-index for validation data
            race_specific_scores = []
            for race in race_groups:
                race_mask = X_val['race_group'] == race
                if sum(race_mask) > 1:  # Only calculate if we have at least 2 samples
                    surv = np.array([(e, t) for e, t in zip(y_event_val[race_mask], y_time_val[race_mask])], 
                                dtype=[('event', bool), ('time', float)])
                    race_cindex = concordance_index_censored(surv['event'], surv['time'], y_pred[race_mask])[0]
                    race_specific_scores.append(race_cindex)
            
            stratified_cindex = np.mean(race_specific_scores) - np.std(race_specific_scores)
            return 'stratified-c-index', stratified_cindex, True
        
        params['objective'] = CoxObjective(y_time_train.values, y_event_train.values)
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=1000,
            feval=cindex_eval,
            callbacks=[lgb.early_stopping(stopping_rounds=100)]
        )
        y_pred = model.predict(X_val)
        
        # Calculate stratified C-index for fold evaluation
        stratified_cindex = cindex_eval(y_pred, X_val)[1]
        fold_scores.append(stratified_cindex)
    
    print('5 folds stratified C-index:', fold_scores)
    return np.mean(fold_scores)

In [5]:
# Now create and run the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3, n_jobs=1)
best_params = study.best_params

[I 2025-02-23 11:27:55,795] A new study created in memory with name: no-name-ac56b860-0188-43dc-8d5a-7ef244c0cbe2


Running in fold 0...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[981]	valid_0's stratified-c-index: 0.670308
Running in fold 1...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's stratified-c-index: 0.665402
Running in fold 2...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[663]	valid_0's stratified-c-index: 0.67158
Running in fold 3...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[996]	valid_0's stratified-c-index: 0.666663
Running in fold 4...
Training until validation scores don't improve for 100 rounds


[I 2025-02-23 11:39:07,378] Trial 0 finished with value: 0.6691694459688596 and parameters: {'num_leaves': 83, 'max_depth': 3, 'learning_rate': 0.04359342334700091, 'reg_alpha': 0.1036044609448446, 'reg_lambda': 0.01075941481680774, 'min_data_in_leaf': 34, 'feature_fraction': 0.9589439313386453, 'bagging_fraction': 0.9146803705656553, 'bagging_freq': 8}. Best is trial 0 with value: 0.6691694459688596.


Did not meet early stopping. Best iteration is:
[984]	valid_0's stratified-c-index: 0.671893
5 folds stratified C-index: [0.6703083210658878, 0.6654016322438691, 0.6715804759564039, 0.6666633419249612, 0.6718934586531767]
Running in fold 0...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[425]	valid_0's stratified-c-index: 0.669396
Running in fold 1...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[189]	valid_0's stratified-c-index: 0.667031
Running in fold 2...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[319]	valid_0's stratified-c-index: 0.673375
Running in fold 3...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[336]	valid_0's stratified-c-index: 0.667561
Running in fold 4...
Training until validation scores don't improve for 100 rounds


[I 2025-02-23 11:43:48,415] Trial 1 finished with value: 0.6700832413326882 and parameters: {'num_leaves': 107, 'max_depth': 8, 'learning_rate': 0.07896707223646482, 'reg_alpha': 0.11605080518387101, 'reg_lambda': 0.025076792784528377, 'min_data_in_leaf': 68, 'feature_fraction': 0.781785417207505, 'bagging_fraction': 0.7804909992546571, 'bagging_freq': 1}. Best is trial 1 with value: 0.6700832413326882.


Early stopping, best iteration is:
[202]	valid_0's stratified-c-index: 0.673053
5 folds stratified C-index: [0.6693963669309454, 0.6670308445651525, 0.6733750811215895, 0.6675610939666392, 0.6730528200791138]
Running in fold 0...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[208]	valid_0's stratified-c-index: 0.667505
Running in fold 1...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[171]	valid_0's stratified-c-index: 0.660858
Running in fold 2...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[104]	valid_0's stratified-c-index: 0.664182
Running in fold 3...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[289]	valid_0's stratified-c-index: 0.670135
Running in fold 4...
Training until validation scores don't improve for 100 rounds


[I 2025-02-23 11:47:09,188] Trial 2 finished with value: 0.6663032527510737 and parameters: {'num_leaves': 94, 'max_depth': 7, 'learning_rate': 0.09084202254308012, 'reg_alpha': 0.0014596453525443948, 'reg_lambda': 0.00024462999665005535, 'min_data_in_leaf': 44, 'feature_fraction': 0.8091222611702042, 'bagging_fraction': 0.7009881182151815, 'bagging_freq': 4}. Best is trial 1 with value: 0.6700832413326882.


Early stopping, best iteration is:
[142]	valid_0's stratified-c-index: 0.668835
5 folds stratified C-index: [0.6675051784323076, 0.6608581406165336, 0.6641824738224218, 0.6701354387564878, 0.6688350321276174]


In [ ]:
# Import necessary libraries for CoxPH
import torch
import torchtuples as tt
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
from sklearn.preprocessing import StandardScaler

# Preprocess the data for CoxPH
# Standardize numerical features
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Convert categorical features to one-hot encoding
X = pd.get_dummies(X, columns=categorical_cols)

# Split the data into training and validation sets using StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
race_groups = X['race_group'].unique()
fold_scores = []

for train_idx, val_idx in skf.split(X, X['race_group']):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_time_train, y_time_val = y_time.iloc[train_idx], y_time.iloc[val_idx]
    y_event_train, y_event_val = y_event.iloc[train_idx], y_event.iloc[val_idx]

    # Prepare the data for CoxPH
    train = (X_train, y_time_train, y_event_train)
    val = (X_val, y_time_val, y_event_val)

    # Define the neural network structure
    num_nodes = [32, 32]
    batch_norm = True
    dropout = 0.1
    output_bias = False

    net = tt.practical.MLPVanilla(X_train.shape[1], num_nodes, batch_norm, dropout, output_bias=output_bias)
    model = CoxPH(net, tt.optim.Adam)

    # Train the model
    batch_size = 256
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience=10)]
    log = model.fit(X_train, (y_time_train, y_event_train), batch_size, epochs, callbacks, val_data=val, val_batch_size=batch_size)

    # Predict survival function for validation data
    surv = model.predict_surv_df(X_val)

    # Evaluate the model using stratified C-index
    race_specific_scores = []
    for race in race_groups:
        race_mask = X_val['race_group'] == race
        if sum(race_mask) > 1:  # Only calculate if we have at least 2 samples
            surv_race = surv.loc[:, race_mask]
            ev = EvalSurv(surv_race, y_time_val[race_mask], y_event_val[race_mask], censor_surv='km')
            race_cindex = ev.concordance_td('antolini')
            race_specific_scores.append(race_cindex)

    stratified_cindex = np.mean(race_specific_scores) - np.std(race_specific_scores)
    fold_scores.append(stratified_cindex)

print('5 folds stratified C-index:', fold_scores)
print('Mean stratified C-index:', np.mean(fold_scores))

KeyError: "None of [Index(['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status',\n       'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe',\n       'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab',\n       'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity',\n       'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe',\n       'prior_tumor', 'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match',\n       'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related',\n       'melphalan_dose', 'cardiac', 'pulm_moderate'],\n      dtype='object')] are in the [columns]"